# 영어/한국어 Word2Vec 실습

### 영어 Word2Vec 만들기

#### 1) 훈련 데이터 이해하기

링크 : https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip

#### 2) 훈련 데이터 전처리하기

In [1]:
import re
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
targetXML = open('C:/Users/it/Downloads/dataset/ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    # 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
    
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [3]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [4]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


#### 3) Word2Vec 훈련시키기

In [5]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
- window = 컨텍스트 윈도우 크기  
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
- workers = 학습을 위한 프로세스 수  
- sg = 0은 CBOW, 1은 Skip-gram.

In [6]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8465383052825928), ('guy', 0.8128888607025146), ('lady', 0.7857546806335449), ('gentleman', 0.7797094583511353), ('boy', 0.7567141056060791), ('girl', 0.7544979453086853), ('kid', 0.7067042589187622), ('soldier', 0.7029512524604797), ('philosopher', 0.653753399848938), ('david', 0.6503821611404419)]


Word2Vec는 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원

#### 4) Word2Vec 모델 저장하고 로드하기

In [7]:
from gensim.models import KeyedVectors

In [8]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model=KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [9]:
model_result=loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8465383052825928), ('guy', 0.8128888607025146), ('lady', 0.7857546806335449), ('gentleman', 0.7797094583511353), ('boy', 0.7567141056060791), ('girl', 0.7544979453086853), ('kid', 0.7067042589187622), ('soldier', 0.7029512524604797), ('philosopher', 0.653753399848938), ('david', 0.6503821611404419)]


### 한국어 Word2Vec 만들기

#### 1) 위키피디아 한국어 덤프 파일 다운로드  

https://dumps.wikimedia.org/kowiki/latest/

file : kowiki-latest-pages-articles.xml.bz2

#### 2) 위키피디아 익스트랙터 다운로드

~~~
git clone "https://github.com/attardi/wikiextractor.git" 
~~~

#### 3) 위키피디아 한국어 덤프 파일 변환

python WikiExtractor.py kowiki-latest-pages-articles.xml.bz2

#### 4) 훈련 데이터 만들기

~~~
copy AA디렉토리의 경로\wiki* wikiAA.txt
copy AB디렉토리의 경로\wiki* wikiAB.txt
copy AC디렉토리의 경로\wiki* wikiAC.txt
copy AD디렉토리의 경로\wiki* wikiAD.txt
copy AE디렉토리의 경로\wiki* wikiAE.txt
copy AF디렉토리의 경로\wiki* wikiAF.txt

copy 현재 디렉토리의 경로\wikiA* wiki_data.txt
~~~

#### 5) 훈련 데이터 전처리 하기

In [16]:
f = open('C:/Users/it/wikiextractor-master/wiki_data.txt', encoding="utf8")

In [17]:
i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 : " %i + line)
    if i == 5:
        break
f.close()

1번째 줄 : <doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 : 지미 카터

3번째 줄 : 제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.

4번째 줄 : 지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

5번째 줄 : 1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.



In [18]:
from konlpy.tag import Okt

In [22]:
# 이모지 제거
def strip_e(st):
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', st)

In [ ]:
okt = Okt()
fread = open('C:/Users/it/wikiextractor-master/wiki_data.txt', encoding="utf8")
n = 0
result = []

In [19]:
while True:
    line = fread.readline()
    if not line: break
    n = n + 1
    
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    line = RE_EMOJI.sub(r'', line)
    
    if n % 5000 == 0: # 5,000의 배수로 While문이 실행될 때마다 몇 번째 While문 실행인지 출력.
        print("%d 번째 while 문 : " %n)
    tokenlist = okt.pos(line, stem=True, norm=True) # 단어 토큰화
    temp = []
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함
    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp)
fread.close()

5000 번째 while 문 : 
10000 번째 while 문 : 
15000 번째 while 문 : 
20000 번째 while 문 : 
25000 번째 while 문 : 
30000 번째 while 문 : 
35000 번째 while 문 : 
40000 번째 while 문 : 
45000 번째 while 문 : 
50000 번째 while 문 : 
55000 번째 while 문 : 
60000 번째 while 문 : 
65000 번째 while 문 : 
70000 번째 while 문 : 
75000 번째 while 문 : 
80000 번째 while 문 : 
85000 번째 while 문 : 
90000 번째 while 문 : 
95000 번째 while 문 : 
100000 번째 while 문 : 
105000 번째 while 문 : 
110000 번째 while 문 : 
115000 번째 while 문 : 
120000 번째 while 문 : 
125000 번째 while 문 : 
130000 번째 while 문 : 
135000 번째 while 문 : 
140000 번째 while 문 : 
145000 번째 while 문 : 
150000 번째 while 문 : 
155000 번째 while 문 : 
160000 번째 while 문 : 
165000 번째 while 문 : 
170000 번째 while 문 : 
175000 번째 while 문 : 
180000 번째 while 문 : 
185000 번째 while 문 : 
190000 번째 while 문 : 
195000 번째 while 문 : 
200000 번째 while 문 : 
205000 번째 while 문 : 
210000 번째 while 문 : 
215000 번째 while 문 : 
220000 번째 while 문 : 
225000 번째 while 문 : 
230000 번째 while 문 : 
235000 번째 while 문 : 
240000 번째 while 문 : 
245000 번째 wh

1920000 번째 while 문 : 
1925000 번째 while 문 : 
1930000 번째 while 문 : 
1935000 번째 while 문 : 
1940000 번째 while 문 : 
1945000 번째 while 문 : 
1950000 번째 while 문 : 
1955000 번째 while 문 : 
1960000 번째 while 문 : 
1965000 번째 while 문 : 
1970000 번째 while 문 : 
1975000 번째 while 문 : 
1980000 번째 while 문 : 
1985000 번째 while 문 : 
1990000 번째 while 문 : 
1995000 번째 while 문 : 
2000000 번째 while 문 : 
2005000 번째 while 문 : 
2010000 번째 while 문 : 
2015000 번째 while 문 : 
2020000 번째 while 문 : 
2025000 번째 while 문 : 
2030000 번째 while 문 : 
2035000 번째 while 문 : 
2040000 번째 while 문 : 
2045000 번째 while 문 : 
2050000 번째 while 문 : 
2055000 번째 while 문 : 
2060000 번째 while 문 : 
2065000 번째 while 문 : 
2070000 번째 while 문 : 
2075000 번째 while 문 : 
2080000 번째 while 문 : 
2085000 번째 while 문 : 
2090000 번째 while 문 : 
2095000 번째 while 문 : 
2100000 번째 while 문 : 
2105000 번째 while 문 : 
2110000 번째 while 문 : 
2115000 번째 while 문 : 
2120000 번째 while 문 : 
2125000 번째 while 문 : 
2130000 번째 while 문 : 
2135000 번째 while 문 : 
2140000 번째 while 문 : 
2145000 번째

3785000 번째 while 문 : 
3790000 번째 while 문 : 
3795000 번째 while 문 : 
3800000 번째 while 문 : 
3805000 번째 while 문 : 
3810000 번째 while 문 : 
3815000 번째 while 문 : 
3820000 번째 while 문 : 
3825000 번째 while 문 : 
3830000 번째 while 문 : 
3835000 번째 while 문 : 
3840000 번째 while 문 : 
3845000 번째 while 문 : 
3850000 번째 while 문 : 
3855000 번째 while 문 : 
3860000 번째 while 문 : 
3865000 번째 while 문 : 
3870000 번째 while 문 : 
3875000 번째 while 문 : 
3880000 번째 while 문 : 
3885000 번째 while 문 : 
3890000 번째 while 문 : 
3895000 번째 while 문 : 
3900000 번째 while 문 : 
3905000 번째 while 문 : 
3910000 번째 while 문 : 
3915000 번째 while 문 : 
3920000 번째 while 문 : 
3925000 번째 while 문 : 
3930000 번째 while 문 : 
3935000 번째 while 문 : 
3940000 번째 while 문 : 
3945000 번째 while 문 : 
3950000 번째 while 문 : 
3955000 번째 while 문 : 
3960000 번째 while 문 : 
3965000 번째 while 문 : 
3970000 번째 while 문 : 
3975000 번째 while 문 : 
3980000 번째 while 문 : 
3985000 번째 while 문 : 
3990000 번째 while 문 : 
3995000 번째 while 문 : 
4000000 번째 while 문 : 
4005000 번째 while 문 : 
4010000 번째

5650000 번째 while 문 : 
5655000 번째 while 문 : 
5660000 번째 while 문 : 
5665000 번째 while 문 : 
5670000 번째 while 문 : 
5675000 번째 while 문 : 
5680000 번째 while 문 : 
5685000 번째 while 문 : 
5690000 번째 while 문 : 
5695000 번째 while 문 : 
5700000 번째 while 문 : 
5705000 번째 while 문 : 
5710000 번째 while 문 : 
5715000 번째 while 문 : 
5720000 번째 while 문 : 
5725000 번째 while 문 : 
5730000 번째 while 문 : 
5735000 번째 while 문 : 
5740000 번째 while 문 : 
5745000 번째 while 문 : 
5750000 번째 while 문 : 
5755000 번째 while 문 : 
5760000 번째 while 문 : 
5765000 번째 while 문 : 
5770000 번째 while 문 : 
5775000 번째 while 문 : 
5780000 번째 while 문 : 
5785000 번째 while 문 : 
5790000 번째 while 문 : 
5795000 번째 while 문 : 
5800000 번째 while 문 : 
5805000 번째 while 문 : 
5810000 번째 while 문 : 
5815000 번째 while 문 : 
5820000 번째 while 문 : 
5825000 번째 while 문 : 
5830000 번째 while 문 : 
5835000 번째 while 문 : 
5840000 번째 while 문 : 
5845000 번째 while 문 : 
5850000 번째 while 문 : 
5855000 번째 while 문 : 
5860000 번째 while 문 : 
5865000 번째 while 문 : 
5870000 번째 while 문 : 
5875000 번째

In [21]:
print('총 샘플의 개수 : {}'.format(len(result))

SyntaxError: unexpected EOF while parsing (<ipython-input-21-791a3cf29425>, line 1)

#### 6) Word2Vec 훈련시키기

In [23]:
from gensim.models import Word2Vec

In [24]:
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [25]:
model_result1=model.wv.most_similar("대한민국")
print(model_result1)

[('한국', 0.6580009460449219), ('조선민주주의인민공화국', 0.5606676340103149), ('우리나라', 0.5238099098205566), ('정인영', 0.5018841028213501), ('경북', 0.5013366341590881), ('이익선', 0.4988616704940796), ('경남', 0.4857853949069977), ('국내', 0.4803791046142578), ('김현준', 0.47913604974746704), ('김지선', 0.4786112904548645)]


In [26]:
model_result2=model.wv.most_similar("어벤져스")
print(model_result2)

[('아이언맨', 0.8587662577629089), ('인크레더블', 0.8172612190246582), ('다크나이트', 0.8114371299743652), ('스파이더맨', 0.8111481070518494), ('솔저', 0.8094265460968018), ('스타트렉', 0.8068864941596985), ('엑스맨', 0.8068361282348633), ('원더우먼', 0.8040205240249634), ('데드풀', 0.7932624816894531), ('퍼니셔', 0.792685866355896)]


In [27]:
model_result3=model.wv.most_similar("반도체")
print(model_result3)

[('전자부품', 0.7843495607376099), ('태양전지', 0.7744534611701965), ('집적회로', 0.7656764984130859), ('웨이퍼', 0.7655747532844543), ('실리콘', 0.7614623308181763), ('그래핀', 0.7364827990531921), ('트랜지스터', 0.7336008548736572), ('팹리스', 0.7266179323196411), ('백열등', 0.7225968837738037), ('하이닉스', 0.7167866826057434)]


### 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding) 소개

여기서는 사전 훈련된 워드 임베딩을 가져와서 간단히 단어들의 유사도를 구해보는 실습을 해보겠습니다. 실제로 모델에 적용해보는 실습은 사전 훈련된 워드 임베딩 챕터에서 진행합니다.

#### 1) 영어

구글이 제공하는 사전 훈련된(미리 학습되어져 있는) Word2Vec 모델을 사용하는 방법

모델 다운로드 경로 :  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit